In [1]:
# import packages
import streamlit as st
import pandas as pd
import numpy as np
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from gensim.models import Doc2Vec
from collections import namedtuple
import gensim.utils
import re
import string
import requests
from sklearn.feature_extraction.text import TfidfVectorizer

from gensim.models import TfidfModel
from gensim.corpora import Dictionary
from gensim import similarities 

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()

In [2]:
# dataframe hasil scraping
df = pd.read_csv(r'Hasil Crawl Halal MUI.csv')
df

,Unnamed: 0,id,no,nama,produsen,sertif,tanggal
0,0,109,21949,cyber park karawaci lt. 2 (head office),"Pt. matahari putra prima, tbk",Hs1a20415/032020/mpp,17-march-2022
1,1,121,21949,foodmart surabaya town square,"Pt. matahari putra prima, tbk",Hs1a20415/032020/mpp,17-march-2022
2,2,95,21949,hypermart batam tanjung pinang,"Pt. matahari putra prima, tbk",Hs1a20415/032020/mpp,17-march-2022
3,3,66,21949,hypermart bau bau,"Pt. matahari putra prima, tbk",Hs1a20415/032020/mpp,17-march-2022
4,4,45,21949,hypermart ciputra world surabaya,"Pt. matahari putra prima, tbk",Hs1a20415/032020/mpp,17-march-2022
...,...,...,...,...,...,...,...
401645,401645,3,7790,장군 olive oil laver,Asia food co.-korea,00100082470417,"08-october-2021"""
401646,401646,4,12758,한상궁 delicious scorched brown rice,"Hando food co., ltd",00100086820118,"25-february-2022"""
401647,401647,2,12758,한상궁 delicous scorched barley and rice,"Hando food co., ltd",00100086820118,"25-february-2022"""
401648,401648,1,12758,한상궁 delicous scorched rice,"Hando food co., ltd",00100086820118,"25-february-2022"""


In [3]:
# hapus nama produk yang terduplikat
df = df.drop_duplicates(subset=['nama'], keep='last')
df

,Unnamed: 0,id,no,nama,produsen,sertif,tanggal
0,0,109,21949,cyber park karawaci lt. 2 (head office),"Pt. matahari putra prima, tbk",Hs1a20415/032020/mpp,17-march-2022
1,1,121,21949,foodmart surabaya town square,"Pt. matahari putra prima, tbk",Hs1a20415/032020/mpp,17-march-2022
2,2,95,21949,hypermart batam tanjung pinang,"Pt. matahari putra prima, tbk",Hs1a20415/032020/mpp,17-march-2022
3,3,66,21949,hypermart bau bau,"Pt. matahari putra prima, tbk",Hs1a20415/032020/mpp,17-march-2022
4,4,45,21949,hypermart ciputra world surabaya,"Pt. matahari putra prima, tbk",Hs1a20415/032020/mpp,17-march-2022
...,...,...,...,...,...,...,...
401645,401645,3,7790,장군 olive oil laver,Asia food co.-korea,00100082470417,"08-october-2021"""
401646,401646,4,12758,한상궁 delicious scorched brown rice,"Hando food co., ltd",00100086820118,"25-february-2022"""
401647,401647,2,12758,한상궁 delicous scorched barley and rice,"Hando food co., ltd",00100086820118,"25-february-2022"""
401648,401648,1,12758,한상궁 delicous scorched rice,"Hando food co., ltd",00100086820118,"25-february-2022"""


In [4]:
@st.cache
def get_data():
    return pd.read_csv("Hasil Crawl Halal MUI.csv")

def cleandata(kalimat):
    kalimatbaru=re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', kalimat)
    kalimatbaru=stopword.remove(kalimatbaru)
    return kalimatbaru

def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    return float(len(s1.intersection(s2)) / len(s1.union(s2)))

In [5]:
lawdata=df
tanya = np.array(df.nama)
jawab=np.array(df.nama+'|'+df.produsen+'|'+df.sertif)
data=np.array(df.nama+' '+df.produsen)

2021-06-13 20:43:05.866 INFO    numexpr.utils: Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2021-06-13 20:43:05.868 INFO    numexpr.utils: NumExpr defaulting to 8 threads.


In [6]:
# fungsi untuk mendapatkan produk dengan nilai jaccard tertinggi
def get_response(q):
	highsim=0.0
	highnum=0
	for i,line in enumerate(data):
		if (type(line)==str):
			qlist=cleandata(q).lower().split()
			linelist=cleandata(line).lower().split()
			simnow=jaccard_similarity(qlist,linelist)
			if(simnow>highsim):
				highnum=i
				highsim=simnow
	return(highnum)  
	

In [7]:
# fungsi untuk mendapatkan nilai jaccard pada produk terpilih
def get_response2(q):
	highsim=0.0
	highnum=0
	for i,line in enumerate(data):
		if (type(line)==str):
			qlist=cleandata(q).lower().split()
			linelist=cleandata(line).lower().split()
			simnow=jaccard_similarity(qlist,linelist)
			if(simnow>highsim):
				highnum=i
				highsim=simnow
	return(highsim)  

In [9]:
# dataframe dari data klikindomaret yang telah diproses
docs2 = pd.read_csv('Data KlikIndomaret Ready.csv')
docs2

,Unnamed: 0,komposisi,produk
0,0,NaN,Pristine 8+ Air Mineral
1,1,NaN,Pristine 8+ Air Mineral
2,2,NaN,Crystalline Air Mineral
3,3,"Calcium, magnesium, sodium, potassium, nitrate...",Le Minerale Air Mineral
4,4,"Calcium, magnesium, sodium, potassium, nitrate...",Le Minerale Air Mineral
...,...,...,...
1328,1328,"Minyak nabati, air, garam, pengemulsi monodigl...",Blue Band Margarine Serbaguna
1329,1329,biji bunga canola.BPOM RI ML 210216001071Diimp...,Dougo Canola Oil
1330,1330,"Minyak nabati, air, garam, pengemulsi monodigl...",Blue Band Margarine Serbaguna
1331,1331,NaN,Blue Band Margarine Cake & Cookie


In [11]:
# hapus kolom "Unnamed: 0"
docs2 = docs2.drop(columns=['Unnamed: 0'])
docs2

,komposisi,Column3
0,NaN,Pristine 8+ Air Mineral
1,NaN,Pristine 8+ Air Mineral
2,NaN,Crystalline Air Mineral
3,"Calcium, magnesium, sodium, potassium, nitrate...",Le Minerale Air Mineral
4,"Calcium, magnesium, sodium, potassium, nitrate...",Le Minerale Air Mineral
...,...,...
1328,"Minyak nabati, air, garam, pengemulsi monodigl...",Blue Band Margarine Serbaguna
1329,biji bunga canola.BPOM RI ML 210216001071Diimp...,Dougo Canola Oil
1330,"Minyak nabati, air, garam, pengemulsi monodigl...",Blue Band Margarine Serbaguna
1331,NaN,Blue Band Margarine Cake & Cookie


In [12]:
# hapus nama produk yang terduplikat
docs2 = docs2.drop_duplicates(keep='last')
docs2

,komposisi,Column3
1,NaN,Pristine 8+ Air Mineral
2,NaN,Crystalline Air Mineral
4,"Calcium, magnesium, sodium, potassium, nitrate...",Le Minerale Air Mineral
7,Air mineral,Club Air Mineral
8,NaN,Aqua Galon Kosong + Isi Ulang
...,...,...
1327,"Air, keju cheddar, pengental pati modifikasi a...",Pro Chiz Keju Gold
1329,biji bunga canola.BPOM RI ML 210216001071Diimp...,Dougo Canola Oil
1330,"Minyak nabati, air, garam, pengemulsi monodigl...",Blue Band Margarine Serbaguna
1331,NaN,Blue Band Margarine Cake & Cookie


In [38]:
# buat list kosong bernama items
items = []

In [39]:
# melakukan pengecekan jaccard similarity secara berulang pada dua dataframe yang telah ada
# hasil tiap perulangan akan disimpan ke dalam list yang telah dibuat disebelumnya
for i, row in docs2.iterrows():
    q1 = row['Column3']
    items.append(q1+"|"+jawab[get_response(q1)]+"|"+str(get_response2(q1)))

In [40]:
print(items)

['Pristine 8+ Air Mineral|Air mineral (aqua)|Sibeste, cv|07340061160719|0.2857142857142857', 'Crystalline Air Mineral|Air mineral (aqua)|Sibeste, cv|07340061160719|0.3333333333333333', 'Le Minerale Air Mineral|Air mineral le minerale|Cv. oto boga jaya|00160117390321|0.5', 'Club Air Mineral|Club air mineral|Pt. trans retail indonesia|00160095720519|0.42857142857142855', 'Aqua Galon Kosong + Isi Ulang|Air isi ulang panji galon|Panji galon|02160016671120|0.375', 'Aqua Air Mineral Click N Go Kids|Air mineral botol aqua|Kampung kecil|00160097550719|0.3', 'Aqua Air Mineral Click N Go|Air mineral botol aqua|Kampung kecil|00160097550719|0.3333333333333333', 'Nestle Air Mineral Pure Life|Air mineral nestle pure life|Pt. kuliner sukses inovatif|00160101680120|0.5555555555555556', 'Ades Air Mineral|Ades air mineral|Pt. trans retail indonesia|00160095720519|0.42857142857142855', 'Aqua Galon|Pegangan galon|Asia, cv.|07170018941013|0.2', 'Aqua Air Mineral|Air mineral botol aqua|Kampung kecil|0016009

In [41]:
# dataframe dari list yang telah dibuat
from pandas import DataFrame
new_df = DataFrame(items,columns=['Produk'])
new_df

,Produk
0,Pristine 8+ Air Mineral|Air mineral (aqua)|Sib...
1,Crystalline Air Mineral|Air mineral (aqua)|Sib...
2,Le Minerale Air Mineral|Air mineral le mineral...
3,Club Air Mineral|Club air mineral|Pt. trans re...
4,Aqua Galon Kosong + Isi Ulang|Air isi ulang pa...
...,...
1276,Pro Chiz Keju Gold|Filling keju defina gold|Cv...
1277,Dougo Canola Oil|Rbd canola oil|Pt. kimia farm...
1278,Blue Band Margarine Serbaguna|Blue band margar...
1279,Blue Band Margarine Cake & Cookie|Blue band ma...


In [42]:
# export ke bentuk csv
new_df.to_csv(r'Match Hasil Jaccard Raw.csv', index = False)

# Processing hasil Jaccard

In [13]:
# dataframe hasil dari jaccard similarity
match = pd.read_csv(r'Match Hasil Jaccard Raw.csv')

# hapus nama produk yang terduplikat
match = match.drop_duplicates()
match

,Produk
0,Pristine 8+ Air Mineral|Air mineral (aqua)|Sib...
1,Crystalline Air Mineral|Air mineral (aqua)|Sib...
2,Le Minerale Air Mineral|Air mineral le mineral...
3,Club Air Mineral|Club air mineral|Pt. trans re...
4,Aqua Galon Kosong + Isi Ulang|Air isi ulang pa...
...,...
1275,Pro Chiz Keju|Keju spesial|Esthree|15160029640...
1276,Pro Chiz Keju Gold|Filling keju defina gold|Cv...
1277,Dougo Canola Oil|Rbd canola oil|Pt. kimia farm...
1278,Blue Band Margarine Serbaguna|Blue band margar...


In [14]:
# membagi kolom produk ke beberapa kolom dengan "|" sebagai pemisah
new = match.Produk.str.split("|", n = 5, expand=True,)

In [15]:
# membuat kolom "produkindo", "produkmui", "produsen", "sertif", dan "nilai"
match["produkindo"]= new[0]
match["produkmui"] = new[1]
match["produsen"] = new[2]
match["sertif"] = new[3]
match["nilai"] = new[4]
match

,Produk,produkindo,produkmui,produsen,sertif,nilai
0,Pristine 8+ Air Mineral|Air mineral (aqua)|Sib...,Pristine 8+ Air Mineral,Air mineral (aqua),"Sibeste, cv",07340061160719,0.2857142857142857
1,Crystalline Air Mineral|Air mineral (aqua)|Sib...,Crystalline Air Mineral,Air mineral (aqua),"Sibeste, cv",07340061160719,0.3333333333333333
2,Le Minerale Air Mineral|Air mineral le mineral...,Le Minerale Air Mineral,Air mineral le minerale,Cv. oto boga jaya,00160117390321,0.5
3,Club Air Mineral|Club air mineral|Pt. trans re...,Club Air Mineral,Club air mineral,Pt. trans retail indonesia,00160095720519,0.42857142857142855
4,Aqua Galon Kosong + Isi Ulang|Air isi ulang pa...,Aqua Galon Kosong + Isi Ulang,Air isi ulang panji galon,Panji galon,02160016671120,0.375
...,...,...,...,...,...,...
1275,Pro Chiz Keju|Keju spesial|Esthree|15160029640...,Pro Chiz Keju,Keju spesial,Esthree,15160029640418,0.2
1276,Pro Chiz Keju Gold|Filling keju defina gold|Cv...,Pro Chiz Keju Gold,Filling keju defina gold,Cv. cahya nusa pratama,01211064181011,0.2
1277,Dougo Canola Oil|Rbd canola oil|Pt. kimia farm...,Dougo Canola Oil,Rbd canola oil,Pt. kimia farma,00080102960320,0.2857142857142857
1278,Blue Band Margarine Serbaguna|Blue band margar...,Blue Band Margarine Serbaguna,Blue band margarine,Pt. trans retail indonesia,00160095720519,0.375


In [16]:
# mengurutkan berdasarkan nilai jaccard similarity tertinggi
match = match.sort_values(['nilai'], ascending=False)
match

,Produk,produkindo,produkmui,produsen,sertif,nilai
487,Dua Kelinci Kacang Sangrai|Dua kelinci kacang ...,Dua Kelinci Kacang Sangrai,Dua kelinci kacang sangrai,Pt. dua kelinci,00100017571001,0.8
483,Dua Kelinci Kacang Polong Polongmas Ayam Bawan...,Dua Kelinci Kacang Polong Polongmas Ayam Bawang,Dua kelinci polongmas kacang polong rasa ayam ...,Pt. dua kelinci,00100017571001,0.7777777777777778
394,Dua Kelinci Jagung Presto Bawang Putih|Dua kel...,Dua Kelinci Jagung Presto Bawang Putih,Dua kelinci jagung presto rasa bawang putih,Pt. dua kelinci,00100017571001,0.75
548,Frisian Flag Susu Bubuk Full Cream|Frisian fla...,Frisian Flag Susu Bubuk Full Cream,Frisian flag susu bubuk full cream,Pt. frisian flag indonesia,00040040520606,0.75
395,Dua Kelinci Jagung Presto Pedas Manis|Dua keli...,Dua Kelinci Jagung Presto Pedas Manis,Dua kelinci jagung presto rasa pedas manis,Pt. dua kelinci,00100017571001,0.75
...,...,...,...,...,...,...
1273,Pro Chiz Keju Slice 5'S|Keju spesial|Esthree|1...,Pro Chiz Keju Slice 5'S,Keju spesial,Esthree,15160029640418,0.14285714285714285
1274,Pro Chiz Keju Slice 10'S|Keju spesial|Esthree|...,Pro Chiz Keju Slice 10'S,Keju spesial,Esthree,15160029640418,0.14285714285714285
558,Tango Drink Don Pedro|Beng beng drink|Rirora d...,Tango Drink Don Pedro,Beng beng drink,Rirora dcafe,05160022181220,0.14285714285714285
331,Milo Healthy Drink Polybag Activ-Go|Dr milo|Dr...,Milo Healthy Drink Polybag Activ-Go,Dr milo,Dr chicken,00160113750121,0.14285714285714285


In [17]:
# export ke bentuk csv
match.to_csv(r'Match Hasil Jaccard Clean.csv')